In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import bilby

import sys
sys.path.insert(0,'../src')
from util import *
from cross_correlation import *
from postprocessing import *
from pe import *
from constants import *

In [ ]:
matplotlib.rcParams.update({'font.size':15})

# Simulate data

In [ ]:
####################
# Inputs
####################

# PSD ~ 1e-42 strain^2 / Hz
# PSD = 2 sigma**2 * deltaT = 2 * sigma**2 / Fs
# = sigma**2 / 512
# sigma = sqrt(1e-42 * 512)

sigma             = np.sqrt(512 * 1e-42) # strain
sigma_inj         = sigma/10.
NSegments         = 50
Fs                = 1024. # Hz
segmentDuration   = 64.0 # s
t0                = 0 # s
TAvg              = 4.0 # s

####################
# Computed quantities
####################

NSamplesPerSegment=int(segmentDuration*Fs) 
deltaT=1/Fs
fNyquist=1/(2*deltaT)
deltaF=1/segmentDuration
deltaFStoch=1/TAvg
NAvgs = 2 * int(segmentDuration / TAvg) - 1
jobDuration = NSegments * segmentDuration

In [ ]:
# Theoretical PSD and sigma for white GWB and white noise

Nfreqs = int((Fs/2) / deltaFStoch)
alpha=3 # white signal
fref=25

w1w2bar, w1w2squaredbar,_,_ = window_factors(NSamplesPerSegment)
PSD_theor = 2*(sigma**2)*deltaT
Pgw_theor = 2*(sigma_inj**2)*deltaT

H_theor = (3*H0**2)/(10*np.pi**2*fref**3) # don't need *(P1.freqs/fref)**(alpha-3) for alpha=3
Y_theor = Pgw_theor/H_theor

var_theor = 1./((segmentDuration)*2*Nfreqs*deltaFStoch * H_theor**2/(PSD_theor**2))
var_theor = w1w2squaredbar / w1w2bar**2 * var_theor
sigma_theor = np.sqrt(var_theor)

print('sigma_theor =', sigma_theor)
print('Y_theor =', Y_theor)

In [ ]:
# simulate the time series data
noise1=sigma*np.random.randn(int(NSamplesPerSegment*NSegments))
noise2=sigma*np.random.randn(int(NSamplesPerSegment*NSegments))
inj=sigma_inj*np.random.randn(int(NSamplesPerSegment*NSegments))

times=np.arange(t0,t0+NSegments*segmentDuration,1/Fs)

d1=TimeSeries(times,noise1 + inj)
d2=TimeSeries(times,noise2 + inj)

# Cross correlation analysis

In [ ]:
# run stochastic pipeline
alpha=0
fref=25
Y_t,sig_t,Y_ft,var_ft,segmentStartTimes,freqs=cross_correlation(d1,d2,segmentDuration,deltaFStoch,
                                                      fref=fref,alpha=alpha,fmin=deltaFStoch,fmax=fNyquist/2.)

In [ ]:
# plot results of CC search
plt.loglog(freqs,np.abs(Y_ft[:,0]),label='Y(f)')
plt.loglog(freqs,np.abs(np.sqrt(var_ft[:,0])),label=('sigma(f)'))
plt.legend()
plt.title('First segment')

In [ ]:
# combine spectra over times

Y_f,var_f=postprocessing_spectra(Y_ft,var_ft,jobDuration,segmentDuration,
                                 deltaFStoch,deltaT)

In [ ]:
# test optimal filtering

alphas=np.linspace(-8,8,100)

snrs=np.zeros(alphas.shape)
for ii,a in enumerate(alphas):
    y,s=calc_Y_sigma_from_Yf_varf(np.real(Y_f),
                          var_f,
                          freqs=freqs,
                          alpha=a,
                          fref=25)
    snrs[ii]=y/s
plt.plot(alphas,snrs)
plt.axvline(3)
plt.xlabel('alpha')
plt.ylabel('SNR')
plt.title('Optimal SNR vs alpha')

# Parameter estimation with power law model

In [ ]:
# A few simple setup steps
label = 'GWB_powerlaw'
outdir = 'outdir'

cleanup_dir(outdir)

Amin,Amax,alpha_min,alpha_max=1e-10,1e-3,-5,5
    

fref=25

likelihood = BasicPowerLawGWBLikelihood(Y_f[1:],var_f[1:],freqs[1:],fref)
priors = dict(A=bilby.core.prior.Uniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.Uniform(alpha_min,alpha_max, 'alpha'))

injection_parameters=dict(A=Y_theor,alpha=3)

# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=500,
    walks=10, outdir=outdir, label=label,maxmcmc=10000,injection_parameters=injection_parameters)
result.plot_corner()

# Parameter estimation with broken power law model

In [ ]:
# A few simple setup steps
label = 'GWB_brokenpowerlaw'
outdir = 'outdir'

cleanup_dir(outdir)

Amin,Amax = 1e-10,1e-3
alpha1_min,alpha1_max=-5,5
alpha2_min,alpha2_max=-5,5
fbreak_min,fbreak_max=30,100
    

fref=25

baselines=[Baseline('test',Y_f,var_f,freqs)]

likelihood = BrokenPowerLawGWBLikelihood(baselines,fref)
priors = dict(A=bilby.core.prior.Uniform(Amin,Amax, 'A'),
              alpha1=bilby.core.prior.Uniform(alpha1_min,alpha1_max, 'alpha1'),
              alpha2=bilby.core.prior.Uniform(alpha2_min,alpha2_max, 'alpha2'),
              fbreak=bilby.core.prior.Uniform(fbreak_min,fbreak_max, 'fbreak'))

injection_parameters=dict(A=Y_theor,alpha1=3,alpha2=3,fbreak=None)


# And run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=500,
    walks=10, outdir=outdir, label=label,maxmcmc=10000,injection_parameters=injection_parameters)
result.plot_corner()

# Model selection

In [ ]:
# Hard coded values from above

# Power law model
lnZ_PL = -1760.873

# Broken power law model
lnZ_BPL = -1763.607

print('lnZ_PL - lnZ_BPL = %f'%(lnZ_PL-lnZ_BPL))